#Testing

## Model Loading

In [ ]:
import numpy as np
from pickle import load
import os
from tqdm import tqdm
from operator import itemgetter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load the dictionaries
with open(r'/content/drive/MyDrive/CS 772/Asg1/new_w2id.pkl', 'rb') as f:
  word2id = load(f)

In [ ]:
# with open(r'/content/drive/MyDrive/CS772-DL for NLP/Dictionaries/new_id2w.pkl', 'rb') as f:
with open(r'/content/drive/MyDrive/CS 772/Asg1/new_id2w.pkl', 'rb') as f:
  id2word = load(f)

In [ ]:
#load SkipGram and CBoW weights
with open(r'/content/drive/MyDrive/CS 772/Asg1/SkipGram.pkl', 'rb') as f:
  SGram = load(f)

SGram_embeddings = SGram['w1']
SGram_embeddings.shape

(51608, 50)

In [ ]:
with open(r'/content/drive/MyDrive/CS 772/Asg1/SkipGram.pkl', 'rb') as f:
  CBoW = load(f)

CBoW_embeddings = CBoW['w1']
CBoW_embeddings.shape

(51608, 50)

In [ ]:
#load validation data
with open('//content/drive/MyDrive/CS 772/Asg1/Validation.txt', 'r') as f:
  instances = f.readlines()

instances = [data.strip('\n') for data in instances]        #remove '\n' from end of each line
print(len(instances))

vocab = list(word2id.keys())
V = len(vocab)
print(V)

991
51608


In [ ]:
#used for one dimensional vectors i.e. with shape (d, )
def softmax(x):
  assert len(x.shape)==1, f"one dimensional ndarray expected. got {len(x.shape)} dim array"
  exp = np.exp(x - np.max(x))
  return exp/exp.sum()

## Validation Set

In [ ]:
top_k = [1, 5, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [ ]:
#validation instances are checked here
#list of predictions and true values in the validation dataset for SGram

SGram_preds = []
SGram_true = []

skipped_instance = []
for instance in tqdm(instances):
  words = [i.lower() for i in instance.split(' ')]

  if words[0] not in vocab:
    a = None
  else:
    a = SGram_embeddings[word2id[words[0]]]

  if words[1] not in vocab:
    b = None
  else:
    b = SGram_embeddings[word2id[words[1]]]

  if words[2] not in vocab:
    c = None
  else:
    c = SGram_embeddings[word2id[words[2]]]


  if a is not None and b is not None and c is not None:
    SGram_true.append(words[-1])
    exp_vec = b - a + c
    dot_prod_args = []
    for i in range(V):
      dot_prod_args.append(np.dot(SGram_embeddings[i], exp_vec))

    prob_dist = softmax(np.array(dot_prod_args))
    prob_dist_indexed = list(enumerate(prob_dist))
    prob_dist_sorted = sorted(prob_dist_indexed, key=itemgetter(1), reverse=True)

    SGram_k = []
    for k in top_k:
      similar_word_id = [tup[0] for tup in prob_dist_sorted[:k]]
      pred_word = []
      for id in similar_word_id:
        pred_word.append(id2word[id])

      SGram_k.append(pred_word)

    SGram_preds.append(SGram_k)
  else:
    skipped_instance.append(instance)


100%|██████████| 991/991 [02:52<00:00,  5.75it/s]


In [ ]:
%%capture
print(f"The following {len(skipped_instance)} instances have been ignored due to one or more words not present in vocab")
for si in skipped_instance:
    print(si)

In [ ]:
print(len(SGram_true), len(SGram_preds), len(SGram_preds[0]))

916 916 11


In [ ]:
#calculate SkipGram accuracy in the validation set
print("SKIPGRAM")
val_num = len(SGram_true)
accuracy_k = []

for i in range(len(top_k)):
  correct = 0
  for j in range(val_num):
    if SGram_true[j] in SGram_preds[j][i]:
      correct += 1

  accuracy = correct/val_num
  print(f'Top-{top_k[i]} -  Correct: {correct} out of {val_num} ; Accuracy: {accuracy}')


SKIPGRAM
Top-5 -  Correct: 7 out of 916 ; Accuracy: 0.007641921397379912
Top-50 -  Correct: 39 out of 916 ; Accuracy: 0.0425764192139738
Top-100 -  Correct: 62 out of 916 ; Accuracy: 0.06768558951965066
Top-150 -  Correct: 74 out of 916 ; Accuracy: 0.08078602620087336
Top-200 -  Correct: 93 out of 916 ; Accuracy: 0.10152838427947598
Top-250 -  Correct: 106 out of 916 ; Accuracy: 0.11572052401746726
Top-300 -  Correct: 122 out of 916 ; Accuracy: 0.1331877729257642
Top-350 -  Correct: 139 out of 916 ; Accuracy: 0.1517467248908297
Top-400 -  Correct: 149 out of 916 ; Accuracy: 0.16266375545851527
Top-450 -  Correct: 161 out of 916 ; Accuracy: 0.17576419213973798
Top-500 -  Correct: 171 out of 916 ; Accuracy: 0.18668122270742357


In [ ]:
#validation instances are checked here
#list of predictions and true values in the validation dataset for SGram

CBoW_preds = []
CBoW_true = []

skipped_instance = []
for instance in tqdm(instances):
  words = [i.lower() for i in instance.split(' ')]

  if words[0] not in vocab:
    a = None
  else:
    a = CBoW_embeddings[word2id[words[0]]]

  if words[1] not in vocab:
    b = None
  else:
    b = CBoW_embeddings[word2id[words[1]]]

  if words[2] not in vocab:
    c = None
  else:
    c = CBoW_embeddings[word2id[words[2]]]


  if a is not None and b is not None and c is not None:
    CBoW_true.append(words[-1])
    exp_vec = b - a + c
    dot_prod_args = []
    for i in range(V):
      dot_prod_args.append(np.dot(CBoW_embeddings[i], exp_vec))

    prob_dist = softmax(np.array(dot_prod_args))
    prob_dist_indexed = list(enumerate(prob_dist))
    prob_dist_sorted = sorted(prob_dist_indexed, key=itemgetter(1), reverse=True)

    CBoW_k = []
    for k in top_k:
      similar_word_id = [tup[0] for tup in prob_dist_sorted[:k]]
      pred_word = []
      for id in similar_word_id:
        pred_word.append(id2word[id])

      CBoW_k.append(pred_word)

    CBoW_preds.append(CBoW_k)
  else:
    skipped_instance.append(instance)

100%|██████████| 991/991 [02:45<00:00,  5.98it/s]


In [ ]:
#calculate SkipGram accuracy in the validation set
print("CBoW")
val_num = len(CBoW_true)
accuracy_k = []

for i in range(len(top_k)):
  correct = 0
  for j in range(val_num):
    if CBoW_true[j] in CBoW_preds[j][i]:
      correct += 1

  accuracy = correct/val_num
  print(f'Top-{top_k[i]} -  Correct: {correct} out of {val_num} ; Accuracy: {accuracy}')

CBoW
Top-5 -  Correct: 7 out of 916 ; Accuracy: 0.007641921397379912
Top-50 -  Correct: 39 out of 916 ; Accuracy: 0.0425764192139738
Top-100 -  Correct: 62 out of 916 ; Accuracy: 0.06768558951965066
Top-150 -  Correct: 74 out of 916 ; Accuracy: 0.08078602620087336
Top-200 -  Correct: 93 out of 916 ; Accuracy: 0.10152838427947598
Top-250 -  Correct: 106 out of 916 ; Accuracy: 0.11572052401746726
Top-300 -  Correct: 122 out of 916 ; Accuracy: 0.1331877729257642
Top-350 -  Correct: 139 out of 916 ; Accuracy: 0.1517467248908297
Top-400 -  Correct: 149 out of 916 ; Accuracy: 0.16266375545851527
Top-450 -  Correct: 161 out of 916 ; Accuracy: 0.17576419213973798
Top-500 -  Correct: 171 out of 916 ; Accuracy: 0.18668122270742357


## Demo

In [ ]:
#code to for test(unseen data)
test_instance = "writer writes reader"
words = test_instance.split(' ')

In [ ]:
Svec1 = SGram_embeddings[word2id[words[0]]]
Svec2 = SGram_embeddings[word2id[words[1]]]
Svec3 = SGram_embeddings[word2id[words[2]]]

S_exp = Svec3 + Svec2 - Svec1

In [ ]:
Cvec1 = CBoW_embeddings[word2id[words[0]]]
Cvec2 = CBoW_embeddings[word2id[words[1]]]
Cvec3 = CBoW_embeddings[word2id[words[2]]]

C_exp = Cvec3 + Cvec2 - Cvec1

In [ ]:
Sgram_args = []
CBoW_args = []

for i in range(V):
  Sgram_args.append(np.dot(SGram_embeddings[i], S_exp))
  CBoW_args.append(np.dot(CBoW_embeddings[i], C_exp))

SGram_prob_dist = softmax(np.array(Sgram_args))
# SGram_similar_word_id = int(np.argmax(SGram_prob_dist))
# SGram_pred_word = id2word[SGram_similar_word_id]
SGram_prob_dist_indexed = list(enumerate(SGram_prob_dist))
SGram_prob_dist_sorted = sorted(SGram_prob_dist_indexed, key=itemgetter(1), reverse=True)

# similar_word_id = int(np.argmax(prob_dist))
SGram_pred_word = []
for k in top_k:
    SGram_similar_word_id = [tup[0] for tup in SGram_prob_dist_sorted[:k]]
    pred_word = []
    for id in SGram_similar_word_id:
        pred_word.append(id2word[id])
    SGram_pred_word.append(pred_word)



CBoW_prob_dist = softmax(np.array(CBoW_args))
CBoW_prob_dist_indexed = list(enumerate(CBoW_prob_dist))
CBoW_prob_dist_sorted = sorted(CBoW_prob_dist_indexed, key=itemgetter(1), reverse=True)

CBoW_pred_word = []
for k in top_k:
    CBoW_similar_word_id = [tup[0] for tup in CBoW_prob_dist_sorted[:k]]
    pred_word = []
    for id in CBoW_similar_word_id:
        pred_word.append(id2word[id])
    CBoW_pred_word.append(pred_word)


print(f"SkipGram : {SGram_pred_word};\nCBoW : {CBoW_pred_word}")

SkipGram : [['counties'], ['counties', 'schengen', 'prakrit', 'lanark', 'myanmar'], ['counties', 'schengen', 'prakrit', 'lanark', 'myanmar', 'communicate', 'townships', 'renfrew', 'weekday', 'characterised', 'russell', 'promoted', 'sisi', 'corresponds', 'coe', 'facto', 'republic', 'eur', 'dhili', 'writes', 'hindi', 'zhou', 'persians', 'guadarrama', 'combe', 'repopulating', 'liveth', 'domestic', 'oecd', 'visaya', 'chuse', 'governorate', 'hominin', 'december', 'verb', 'eurozone', 'el', 'walking', 'questions', 'contribution', 'amorites', 'parliamentary', 'vatican', 'code', 'les', 'china', 'nato', 'thos', 'properly', 'identification'], ['counties', 'schengen', 'prakrit', 'lanark', 'myanmar', 'communicate', 'townships', 'renfrew', 'weekday', 'characterised', 'russell', 'promoted', 'sisi', 'corresponds', 'coe', 'facto', 'republic', 'eur', 'dhili', 'writes', 'hindi', 'zhou', 'persians', 'guadarrama', 'combe', 'repopulating', 'liveth', 'domestic', 'oecd', 'visaya', 'chuse', 'governorate', 'hom